In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
#import tensorflow_text as text
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from keras.models import load_model, Sequential
from keras.layers import Embedding, Dense, SimpleRNN
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import string
import re

In [4]:
# Load data
data = pd.read_csv('../data/raw/filtered.tsv', sep='\t')
data.head()

,Unnamed: 0,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
0,0,"If Alkar is flooding her with psychic waste, t...","if Alkar floods her with her mental waste, it ...",0.785171,0.010309,0.014195,0.981983
1,1,Now you're getting nasty.,you're becoming disgusting.,0.749687,0.071429,0.065473,0.999039
2,2,"Well, we could spare your life, for one.","well, we can spare your life.",0.919051,0.268293,0.213313,0.985068
3,3,"Ah! Monkey, you've got to snap out of it.","monkey, you have to wake up.",0.664333,0.309524,0.053362,0.994215
4,4,I've got orders to put her down.,I have orders to kill her.,0.726639,0.181818,0.009402,0.999348


In [8]:
data = data.iloc[:10000]

In [9]:
def preprocess_sentence(s, put_tok=True):
    s = s.lower().strip()
    text = re.sub(r'[0-9]', '' ,s)
    tokens = nltk.word_tokenize(s)
    tokens = [word for word in tokens if (word not in stopwords.words()) and (len(word) > 2) \
              and (word not in string.punctuation)]

    if put_tok:
        s = 'sos ' + ' '.join(tokens) + ' eos'
    return s

In [10]:
data['reference'] = data['reference'].apply(lambda x: preprocess_sentence(x, put_tok=False))
data['translation'] = data['translation'].apply(preprocess_sentence)

In [11]:
data.head()

,Unnamed: 0,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
0,0,"if alkar is flooding her with psychic waste, t...",sos alkar floods mental waste explain high lev...,0.785171,0.010309,0.014195,0.981983
1,1,now you're getting nasty.,sos 're disgusting eos,0.749687,0.071429,0.065473,0.999039
2,2,"well, we could spare your life, for one.",sos spare life eos,0.919051,0.268293,0.213313,0.985068
3,3,"ah! monkey, you've got to snap out of it.",sos monkey wake eos,0.664333,0.309524,0.053362,0.994215
4,4,i've got orders to put her down.,sos orders kill eos,0.726639,0.181818,0.009402,0.999348


In [12]:
def tokenize(lang):
    lang_tokenizer = Tokenizer()
    lang_tokenizer.fit_on_texts(lang)

    tensor = lang_tokenizer.texts_to_sequences(lang)
    tensor = pad_sequences(tensor, padding='post')

    return tensor, lang_tokenizer

reference_tensor, ref_lang = tokenize(data['reference'])
translation_tensor, tran_lang = tokenize(data['translation'])

max_length_targ, max_length_inp = translation_tensor.shape[1], reference_tensor.shape[1]

In [13]:
reference_tensor_train, reference_tensor_val, translation_tensor_train, translation_tensor_val = train_test_split(reference_tensor, translation_tensor, test_size=0.2)

In [16]:
reference_tensor_train.shape

(8000, 180)

In [17]:
data.to_csv("Data_preprocess.csv")
pd.DataFrame(reference_tensor_train).to_csv("Reference_train.csv")
pd.DataFrame(reference_tensor_val).to_csv("Reference_val.csv")
pd.DataFrame(translation_tensor_train).to_csv("Translation_train.csv")
pd.DataFrame(translation_tensor_val).to_csv("Translation_val.csv")

In [18]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, \
    Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np